In [ ]:
import random, numpy as np, argparse
from types import SimpleNamespace

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from bert import BertModel
from optimizer import AdamW
from tqdm import tqdm
from datasets import (
    SentenceClassificationDataset,
    SentenceClassificationTestDataset,
    SentencePairDataset,
    SentencePairTestDataset,
    load_multitask_data,
)

from evaluation import (
    model_eval_sst,
    model_eval_multitask,
    model_eval_test_multitask,
    model_val_sts,
    model_val_para,
)

In [ ]:
# prepare dataset
sst_train, sst_dev = "data/ids-sst-train.csv", "data/ids-sst-dev.csv"
para_train, para_dev = "data/quora-train.csv", "data/quora-dev.csv"
sts_train, sts_dev = "data/sts-train.csv", "data/sts-dev.csv"


sst_train_data, num_labels, para_train_data, sts_train_data = load_multitask_data(
    sst_train, para_train, sts_train, split="train"
)
sst_dev_data, num_labels, para_dev_data, sts_dev_data = load_multitask_data(
    sst_dev, para_dev, sts_dev, split="dev"
)

sst_train_data = SentenceClassificationDataset(sst_train_data, None)
sst_dev_data = SentenceClassificationDataset(sst_dev_data, None)


sst_train_dataloader = DataLoader(
    sst_train_data,
    shuffle=True,
    batch_size=8,
    collate_fn=sst_train_data.collate_fn,
)
sst_dev_dataloader = DataLoader(
    sst_dev_data,
    shuffle=False,
    batch_size=8,
    collate_fn=sst_dev_data.collate_fn,
)

para_train_data = SentencePairDataset(para_train_data, None)
para_dev_data = SentencePairDataset(para_dev_data, None)

para_train_dataloader = DataLoader(
    para_train_data,
    shuffle=True,
    batch_size=32,
    collate_fn=para_train_data.collate_fn,
)
para_dev_dataloader = DataLoader(
    para_dev_data,
    shuffle=False,
    batch_size=32,
    collate_fn=para_dev_data.collate_fn,
)

sts_train_data = SentencePairDataset(sts_train_data, None, isRegression=True)
sts_dev_data = SentencePairDataset(sts_dev_data, None, isRegression=True)

sts_train_dataloader = DataLoader(
    sts_train_data,
    shuffle=True,
    batch_size=8,
    collate_fn=sts_train_data.collate_fn,
)
sts_dev_dataloader = DataLoader(
    sts_dev_data,
    shuffle=False,
    batch_size=8,
    collate_fn=sts_dev_data.collate_fn,
)

In [ ]:
# define model
from multitask_classifier import MultitaskBERT

if torch.cuda.is_available():
    device = torch.device(f"cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
config = SimpleNamespace(
    **{
        "hidden_dropout_prob": 0.3,
        "num_labels": num_labels,
        "hidden_size": 768,
        "data_dir": ".",
        "fine_tune_mode": "full-model",
    }
)
# model_path = (
#     "./predictions/single_task_model/single_sts/full-model-20-1e-05-multitask.pt"
# )
model_path = ""
model = MultitaskBERT(config)
if model_path:
    saved = torch.load(model_path, map_location=device)
    config = saved["model_config"]
    model.load_state_dict(saved["model"])
    print(f"using fine-tuned BERT in {device}!")
else:
    print(f"using pretrained NOT fine-tuned base BERT in {device}!")

model = model.to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"model has {total_params} parameters!")


In [ ]:
# calculate and generate list of cls embeddings and avg tken embedding
import pandas as pd

sst_dataloader, para_dataloader, sts_dataloader = (
    sst_dev_dataloader,
    para_dev_dataloader,
    sts_dev_dataloader,
)
sts_cos_df = pd.DataFrame(
    index=range(len(sts_dev_data)), columns=["id", "label", "cls_cos", "avg_cos"]
)
sts_cos_df["label"].astype(np.float16)
ridx = 0
for batch in tqdm(sts_dataloader):
    tk_id1 = batch["token_ids_1"].to(device)
    at_msk1 = batch["attention_mask_1"].to(device)
    tken_type_ids_1 = batch["token_type_ids_1"].to(device)
    tk_id2 = batch["token_ids_2"].to(device)
    at_msk2 = batch["attention_mask_2"].to(device)
    tken_type_ids_2 = batch["token_type_ids_2"].to(device)
    labels = batch["labels"].float().to(device)
    sents_id = batch["sent_ids"]
    btch_sz = tk_id1.shape[0]
    # get the embeddings
    sent1_cls, sent1_avg = model.forward(tk_id1, at_msk1)
    sent2_cls, sent2_avg = model.forward(tk_id2, at_msk2)

    cls_cos = F.cosine_similarity(sent1_cls, sent2_cls)
    avg_cos = F.cosine_similarity(sent1_avg, sent2_avg)
    sts_cos_df.iloc[ridx : ridx + btch_sz, 2:4] = np.concatenate(
        (
            cls_cos.detach().cpu().numpy()[:, np.newaxis],
            avg_cos.detach().cpu().numpy()[:, np.newaxis],
        ),
        axis=1,
    )

    sts_cos_df.iloc[ridx : ridx + btch_sz, 0] = sents_id
    sts_cos_df.iloc[ridx : ridx + btch_sz, 1] = labels.cpu()
    ridx += btch_sz

In [ ]:
# Rename the df
# sgl_sts_BERT_sts_df = sts_cos_df.copy()
pretr_BERT_sts_df = sts_cos_df.copy()

In [ ]:
# some visualization of pretrained BERT
import seaborn as ses
from matplotlib import pyplot as plt

sgl_sts_BERT_sts_df["label_cat"] = sgl_sts_BERT_sts_df["label"].astype(int)
pretr_BERT_sts_df["label_cat"] = pretr_BERT_sts_df["label"].astype(int)
xl = (-0.5, 5.5)
f = plt.figure(figsize=(10, 10))
plt.subplot(3, 1, 1)
ax = ses.histplot(pretr_BERT_sts_df["label"], bins=25, color="tab:orange")
ax.set_xlim(xl)
ax.set_xticks(range(6))
plt.subplot(3, 2, 3)
ax = ses.histplot(pretr_BERT_sts_df["cls_cos"] * 5, bins=25, color="tab:green")
ax.set_xlim(xl)
ax.set_xticks(range(6))
plt.subplot(3, 2, 4)
ax = ses.histplot(pretr_BERT_sts_df["avg_cos"] * 5, bins=25, color="tab:blue")
ax.set_xlim(xl)
ax.set_ylabel("")
ax.set_xticks(range(6))
plt.subplot(3, 2, 5)
ax = ses.histplot(sgl_sts_BERT_sts_df["cls_cos"] * 5, bins=25, color="tab:green")
ax.set_xlim(xl)
ax.set_xticks(range(6))
plt.subplot(3, 2, 6)
ax = ses.histplot(sgl_sts_BERT_sts_df["avg_cos"] * 5, bins=25, color="tab:blue")
ax.set_xlim(xl)
ax.set_xticks(range(6))

In [ ]:
# summarize the cos and bins into a summary df
# plot bar plot along cosine axis, for each model output
f, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
f1 = ses.scatterplot(
    data=pretr_BERT_sts_df, x="label", y="cls_cos", hue="label_cat", ax=ax[0, 0]
)
f1.get_legend().set_visible(False)
f1.set_xlim(xl)
f1.set_xticks(range(6))
f2 = ses.scatterplot(
    data=pretr_BERT_sts_df, x="label", y="avg_cos", hue="label_cat", ax=ax[0, 1]
)
f2.get_legend().set_visible(False)
f2.set_xlim(xl)
f2.set_xticks(range(6))
f3 = ses.scatterplot(
    data=sgl_sts_BERT_sts_df, x="label", y="cls_cos", hue="label_cat", ax=ax[1, 0]
)
f3.get_legend().set_visible(False)
f3.set_xlim(xl)
f3.set_xticks(range(6))
f4 = ses.scatterplot(
    data=sgl_sts_BERT_sts_df, x="label", y="avg_cos", hue="label_cat", ax=ax[1, 1]
)
f4.get_legend().set_visible(False)
f4.set_xlim(xl)
f4.set_xticks(range(6))

In [ ]:
# plot the delta cos similarity
sgl_sts_BERT_sts_df["dcos_cls"] = (
    sgl_sts_BERT_sts_df["cls_cos"] - pretr_BERT_sts_df["cls_cos"]
)
sgl_sts_BERT_sts_df["dcos_avg"] = (
    sgl_sts_BERT_sts_df["avg_cos"] - pretr_BERT_sts_df["avg_cos"]
)

f, ax = plt.subplots(nrows=2, ncols=1, figsize=(6, 8))
f1 = ses.histplot(
    data=sgl_sts_BERT_sts_df, x="dcos_cls", hue="label_cat", ax=ax[0], bins=25
)
# f1.get_legend().set_visible(False)
f1.set_xlim((-1.2, 1.2))
f1 = ses.histplot(
    data=sgl_sts_BERT_sts_df, x="dcos_avg", hue="label_cat", ax=ax[1], bins=25
)
f1.set_xlim((-1.2, 1.2))
# f1.get_legend().set_visible(False)